## Simple draw axis program

A quick example on how to render an axis in the scene

In [1]:
import ipywebgl
import numpy as np

w = ipywebgl.GLViewer()
w.clear_color(.8, .8, .8 ,1)
w.clear()
w.enable(depth_test=True)
w.execute_commands(execute_once=True)


axis_prog = w.create_program_ext(
'''#version 300 es
    //the ViewBlock that is automatically filled by ipywebgl
    layout(std140) uniform ViewBlock
    {
        mat4 u_cameraMatrix;          //the camera matrix in world space
        mat4 u_viewMatrix;            //the inverse of the camera matrix
        mat4 u_projectionMatrix;      //the projection matrix
        mat4 u_viewProjectionMatrix;  //the projection * view matrix
    };

    uniform float u_scale;
    in vec3 in_vert;
    in vec3 in_color;
    out vec3 v_color;    
    void main() {
        gl_Position = u_viewProjectionMatrix * vec4(in_vert * u_scale, 1.0);
        v_color = in_color;
    }
''',
'''#version 300 es
    precision highp float;
    in vec3 v_color;
    out vec4 f_color;
    void main() {
        f_color = vec4(v_color, 1.0);
    }
''')

axis_vbo = w.create_buffer_ext(
    src_data= np.array([
            # x, y ,z red, green, blue
            0, 0, 0, 1, 0, 0,
            1, 0, 0, 1, 0, 0,
            0, 0, 0, 0, 1, 0,
            0, 1, 0, 0, 1, 0,
            0, 0, 0, 0, 0, 1,
            0, 0, 1, 0, 0, 1,
        ], dtype=np.float32)
)

axis_vao = w.create_vertex_array_ext(
    axis_prog,
    [
        (axis_vbo, '3f32 3f32', 'in_vert', 'in_color'),
    ]
)

w.clear()
w.use_program(axis_prog)
w.uniform('u_scale', np.array([10.0], dtype=np.float32))
w.bind_vertex_array(axis_vao)
w.draw_arrays('LINES', 0, 6)
# render in loop if needed
w.execute_commands()
w

GLViewer(camera_pos=[0, 50, 200])